In [80]:
import iris
import matplotlib

import matplotlib.pyplot as plt
import iris.plot as iplt




In [28]:
cmip_path = "/g/data/oi10/replicas/CMIP6/CMIP/NCC/NorESM2-MM/historical/r1i1p1f1/6hrLev/"
barpa_path = "/g/data/tp28/dev/barpa/prod/chs548/cg282_NorESM2-MM_ssp370_2014_sciB/20140101T0000Z/nc/"

In [63]:
cy = iris.Constraint(latitude=lambda x: -20<=x<=0)
cx = iris.Constraint(longitude=lambda x: 160<=x<=200)
ct = iris.Constraint(time=lambda t: (t.point.year,t.point.month,t.point.day)in [(2014,1,1),(2013,12,31)])

In [64]:
cmip = iris.load([cmip_path+var+"/*/*/*201001010300-201412312100.nc" for var in ['ta','ps']],cx&cy&ct)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'ps'
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'ta'
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/iris/fileformats/cf.py:1205: UserWarning: Ignoring formula terms variable 'ps' referenced by data variable 'b_bnds' via variable 'lev': Dimensions ('time', 'lat', 'lon') do not span ('lev', 'bnds')
  warnings.warn(msg)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/iris/fileformats/cf.py:1205: UserWarning: Ignoring formula terms variable 'ps' referenced by data variable 'a_bnds' via variable 'lev': Dimensions ('time', 'lat', 

In [57]:
barpa=iris.load([barpa_path+"/PRS*H/"+var+"-CMIP6-NorESM2-MM-ssp370-r1i1p1f1-barpa_r-v1-201401010000-201401010000.nc" for var in ['air_temp_uv']],cx&cy)

In [17]:
print(cmip[0].coord('time'),barpa[0].coord('time'))

DimCoord([2014-01-01 03:00:00, 2014-01-01 09:00:00, 2014-01-01 15:00:00,
       2014-01-01 21:00:00], standard_name='time', calendar='365_day', long_name='time', var_name='time') DimCoord([2014-01-01 03:00:00, 2014-01-01 06:00:00, 2014-01-01 09:00:00,
       2014-01-01 12:00:00, 2014-01-01 15:00:00, 2014-01-01 18:00:00,
       2014-01-01 21:00:00, 2014-01-02 00:00:00], standard_name='time', calendar='365_day', long_name='time', var_name='time')


In [23]:
print(cmip[0].coord('longitude')[-1],barpa[0].coord('longitude')[-1])

DimCoord(array([200.]), bounds=array([[199.375, 200.625]]), standard_name='longitude', units=Unit('degrees'), long_name='Longitude', var_name='lon') DimCoord(array([199.9707489]), bounds=array([[199.89350128, 200.04799652]]), standard_name='longitude', units=Unit('degrees'), long_name='longitude', var_name='longitude')


In [69]:
cmip_aw = iris.analysis.cartography.area_weights(cmip[1])
#barpa[0].coord('longitude').guess_bounds()
#barpa[0].coord('latitude').guess_bounds()
barpa_aw = iris.analysis.cartography.area_weights(barpa[0])

/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")
/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")


In [81]:
iplt.plot(cmip[1][3:5].collapsed(['time','latitude','longitude'],iris.analysis.MEAN,weights=cmip_aw[3:5]),cmip[-1][3:5].collapsed(['time','latitude','longitude'],iris.analysis.MEAN,weights=cmip_aw[3:5]),'.-')
iplt.plot(barpa[0][0].collapsed(['latitude','longitude'],iris.analysis.MEAN,weights=barpa_aw[0]),'.-')#,barpa[1].collapsed(['time','latitude','longitude'],iris.analysis.MEAN,weights=barpa_aw).data)
#plt.ylim(300,10)
#plt.xlim(295,301)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/iris/coords.py:1982: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/iris/coords.py:1976: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'vertical coordinate formula term: ps'.
  warnings.warn(msg.format(self.name()))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/iris/aux_factory.py:119: UserWarning: Surface pressure coordinate 'vertical coordinate formula term: ps' has bounds. These will be disregarded.
  self._check_dependencies(**new_dependencies)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/iris/coords.py:1982: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descript

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [65]:
cmip

[<iris 'Cube' of vertical coordinate formula term: ps / (Pa) (time: 8; latitude: 23; longitude: 33)>,
<iris 'Cube' of air_temperature / (K) (time: 8; atmosphere_hybrid_sigma_pressure_coordinate: 32; latitude: 23; longitude: 33)>,
<iris 'Cube' of surface_air_pressure / (Pa) (time: 8; latitude: 23; longitude: 33)>]

In [33]:
print(cmip[1])

air_temperature / (K)                                        (time: 4; atmosphere_hybrid_sigma_pressure_coordinate: 32; latitude: 23; longitude: 33)
    Dimension coordinates:
        time                                                      x                                               -             -              -
        atmosphere_hybrid_sigma_pressure_coordinate               -                                               x             -              -
        latitude                                                  -                                               -             x              -
        longitude                                                 -                                               -             -              x
    Auxiliary coordinates:
        vertical coordinate formula term: ps                      x                                               -             x              x
        vertical coordinate formula term: a(k)                    -     

In [66]:
cmip.append(cmip[1].copy(data=cmip[1].coord('air_pressure').points/100))
cmip[-1].units = "hPa"
print(cmip)

0: vertical coordinate formula term: ps / (Pa) (time: 8; latitude: 23; longitude: 33)
1: air_temperature / (K)               (time: 8; atmosphere_hybrid_sigma_pressure_coordinate: 32; latitude: 23; longitude: 33)
2: surface_air_pressure / (Pa)         (time: 8; latitude: 23; longitude: 33)
3: air_temperature / (hPa)             (time: 8; atmosphere_hybrid_sigma_pressure_coordinate: 32; latitude: 23; longitude: 33)


In [43]:
print(barpa[0].coord('pressure'))

DimCoord(array([ 200.,  300.,  400.,  500.,  600.,  700.,  850.,  925.,  950.,
       1000.]), standard_name=None, units=Unit('hPa'), long_name='pressure', var_name='pressure')


[<iris 'Cube' of air_temperature / (K) (time: 1; pressure: 10; latitude: 130; longitude: 259)>,
<iris 'Cube' of air_temperature / (K) (time: 1; pressure: 8; latitude: 130; longitude: 259)>]